<a href="https://colab.research.google.com/github/Ronin1289/Price-Predictor/blob/main/Price_Predictor_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bidirectional LSTM

##Importing Libraries

In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=f1aabea5d60e72b223c2ba6f973acb8de9f50c8d6b14f43b64f70ffe0387d105
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


##Loading Data

In [ ]:
def load_data(ticker, start, end, filename="BTC-USD_data.csv"):
    df = yf.download(ticker, start=start, end=end, auto_adjust=False)  # Ensure raw price data
    df['RSI'] = ta.momentum.RSIIndicator(pd.Series(df['Close'].squeeze()), window=14).rsi()  # Compute RSI
    df['RSI'] = df['RSI'].astype(float)  # Convert to 1D Series if needed
    df['Trend'] = df['Close'].pct_change().fillna(0)  # Compute trend
    df.dropna(inplace=True)  # Remove missing values

    # Save to CSV
    df.to_csv(filename, index=True)
    print(f"Data saved to {filename}")

    return df

# Load and save data
data = load_data("BTC-USD", start="2020-01-01", end="2024-01-01", filename="BTC-USD_data.csv")


[*********************100%***********************]  1 of 1 completed

Data saved to BTC-USD_data.csv


In [ ]:
data.head()

Price,Adj Close,Close,High,Low,Open,Volume,RSI,Trend
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,,
Date,,,,,,,,
2020-01-14,8827.764648,8827.764648,8879.511719,8140.933105,8140.933105,44841784107,79.417731,0.083933
2020-01-15,8807.010742,8807.010742,8890.117188,8657.187500,8825.343750,40102834650,78.574650,-0.002351
2020-01-16,8723.786133,8723.786133,8846.460938,8612.095703,8812.481445,31313981931,75.130328,-0.009450
2020-01-17,8929.038086,8929.038086,8958.122070,8677.316406,8725.209961,36372139320,77.723806,0.023528
2020-01-18,8942.808594,8942.808594,9012.198242,8827.332031,8927.211914,32337772627,77.890392,0.001542


##Preparing Data

In [ ]:
# 2. Prepare Data for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, 0])  # Predicting close price
    return np.array(X), np.array(y)

# Load data
# data = load_data("AAPL", start="2020-01-01", end="2024-01-01")

# Normalize features
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['Close', 'RSI', 'Volume', 'Trend']])

# Create sequences
seq_length = 100  # Using past 50 days
dataset_X, dataset_y = create_sequences(data_scaled, seq_length)

# Split data
train_size = int(len(dataset_X) * 0.8)
X_train, X_test = dataset_X[:train_size], dataset_X[train_size:]
y_train, y_test = dataset_y[:train_size], dataset_y[train_size:]


##Building Model

In [ ]:
from tensorflow.keras.layers import Bidirectional

model = Sequential([
    Bidirectional(LSTM(100, return_sequences=True), input_shape=(seq_length, 4)),
    Dropout(0.3),
    Bidirectional(LSTM(50, return_sequences=False)),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


##Training Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Save weights after every epoch
checkpoint = ModelCheckpoint(
    "model_epoch_{epoch:02d}.weights.h5",
    save_weights_only=True,
    save_freq='epoch'
)
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test),callbacks=[checkpoint])


Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 15s 223ms/step - loss: 0.0011 - val_loss: 2.0923e-04
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 15s 225ms/step - loss: 0.0010 - val_loss: 2.3023e-04
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 21s 226ms/step - loss: 0.0011 - val_loss: 2.2862e-04
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 223ms/step - loss: 0.0012 - val_loss: 4.6718e-04
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 223ms/step - loss: 0.0011 - val_loss: 1.9945e-04
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 16s 235ms/step - loss: 0.0011 - val_loss: 2.2568e-04
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 232ms/step - loss: 0.0010 - val_loss: 2.3723e-04
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 15s 225ms/step - loss: 9.9778e-04 - val_loss: 5.0450e-04
Epoch 9/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 224ms/step - loss: 0.0013 - val_loss: 2.3419e-04
Epoch 10/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 218ms/step - loss: 0.0012 - val_loss: 2.2480e-04
Epoch 11/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 15s 225ms/step - loss: 8.6599e-04 -

##Analyzing Performance

In [ ]:
print(f"Model Accuracy (R² Score): {r2:.4f}")

Model Accuracy (R² Score): 0.9759


In [ ]:
epoch_to_load = 50-2
weights_path = f"model_epoch_{epoch_to_load:02d}.weights.h5"
model.load_weights(weights_path)

print(f"Loaded weights from epoch {epoch_to_load}")


Loaded weights from epoch 48


In [ ]:
from sklearn.metrics import r2_score

# Get model predictions
y_pred = model.predict(X_test)

# Inverse transform to get actual price values
y_test_actual = scaler.inverse_transform(np.column_stack((y_test, np.zeros((len(y_test), 3)))))[:, 0]
y_pred_actual = scaler.inverse_transform(np.column_stack((y_pred.flatten(), np.zeros((len(y_pred), 3)))))[:, 0]

# Compute R² Score
r2 = r2_score(y_test_actual, y_pred_actual)

# Print R² Score (Accuracy)
print(f"Model Accuracy (R² Score): {r2:.4f}")

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step
Model Accuracy (R² Score): 0.9759


In [ ]:
from sklearn.metrics import r2_score

# Baseline prediction: Yesterday's closing price as today's prediction
y_baseline = X_test[:, -1, 0]  # Last known "Close" price in each sequence

# Compare naive prediction vs actual
r2_baseline = r2_score(y_test, y_baseline)
r2_model = r2_score(y_test, y_pred)

# Print evaluation
if r2_model > r2_baseline:
    print("✅ Model is performing better than the baseline.")
else:
    print("❌ Model is worse than simply guessing the previous close price.")


❌ Model is worse than simply guessing the previous close price.


In [ ]:
print(r2_baseline)

0.9846805957006914


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Evaluate training and validation loss
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.title("Training & Validation Loss")
plt.show()

# Get predictions
y_pred = model.predict(X_test)

# Inverse transform to get actual price values
y_test_actual = scaler.inverse_transform(np.column_stack((y_test, np.zeros((len(y_test), 3)))))[:, 0]
y_pred_actual = scaler.inverse_transform(np.column_stack((y_pred.flatten(), np.zeros((len(y_pred), 3)))))[:, 0]

# Compute Evaluation Metrics
mae = mean_absolute_error(y_test_actual, y_pred_actual)
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
r2 = r2_score(y_test_actual, y_pred_actual)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

# Plot Actual vs Predicted
plt.figure(figsize=(10, 5))
plt.plot(y_test_actual, label="Actual Price", color='blue')
plt.plot(y_pred_actual, label="Predicted Price", color='red', linestyle="dashed")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.legend()
plt.title("Actual vs Predicted Stock Price")
plt.show()


KeyError: 'loss'